In [ ]:
# Install required packages
%pip install google-play-scraper pandas --quiet

from google_play_scraper import Sort, reviews, app
import pandas as pd
from datetime import datetime, timedelta
import time

# App package names
safaricom_app_id = 'com.safaricom.mysafaricom'
airtel_app_id = 'com.airtel.africa.selfcare'

# Configuration
MIN_REVIEWS_PER_APP = 2500  # To ensure we get 5,000 total reviews (2,500 per app)
START_DATE = datetime(2022, 1, 1).timestamp()  # Start from January 1, 2022
all_reviews = []

def get_app_reviews(app_id, app_name):
    print(f"Fetching reviews for {app_name}...")
    review_count = 0
    continuation_token = None
    
    try:
        # First, get app details
        app_details = app(app_id, country='ke', lang='en')
        print(f"\nApp Details for {app_name}:")
        print(f"Current Rating: {app_details['score']}")
        print(f"Total Reviews: {app_details['reviews']}")
        print(f"Last Updated: {datetime.fromtimestamp(app_details['updated']).strftime('%Y-%m-%d')}\n")
    except Exception as e:
        print(f"Error fetching app details: {str(e)}")

    while True:
        try:
            # Add delay to avoid rate limiting
            time.sleep(1)
            
            # Fetch a batch of reviews
            result, continuation_token = reviews(
                app_id,
                lang='en',        # Language (English)
                country='ke',     # Country (Kenya)
                sort=Sort.NEWEST, # Sort by newest first
                count=100,        # Number of reviews per batch (max is 100)
                continuation_token=continuation_token
            )
            
            # Filter and process reviews
            current_batch = []
            for review in result:
                # Handle different timestamp formats
                review_at = review['at']
                if isinstance(review_at, datetime):
                    review_timestamp = review_at.timestamp()
                else:
                    review_timestamp = review_at
                
                # Check if review is within our date range (2022 onwards)
                if review_timestamp < START_DATE:
                    print(f"\nReached reviews older than 2022 for {app_name}")
                    if current_batch:  # Add any remaining valid reviews
                        all_reviews.extend(current_batch)
                        review_count += len(current_batch)
                    return review_count
                
                # Add additional information
                review['app_name'] = app_name
                review['scrape_date'] = datetime.now().strftime('%Y-%m-%d')
                review['review_date'] = datetime.fromtimestamp(review_timestamp).strftime('%Y-%m-%d')
                current_batch.append(review)
            
            # Add batch to all reviews
            all_reviews.extend(current_batch)
            review_count += len(current_batch)
            print(f"Fetched {review_count} reviews for {app_name}")
            
            # Break if we have enough reviews or no more are available
            if review_count >= MIN_REVIEWS_PER_APP or continuation_token is None:
                break
                
        except Exception as e:
            print(f"Error fetching reviews: {str(e)}")
            time.sleep(5)  # Wait longer if there's an error
            continue

    return review_count

# Fetch reviews for both apps
print("Starting review collection...")
print(f"Collecting reviews from {datetime.fromtimestamp(START_DATE).strftime('%Y-%m-%d')} to present...")
total_safaricom = get_app_reviews(safaricom_app_id, 'Safaricom App')
total_airtel = get_app_reviews(airtel_app_id, 'Airtel Money App')

# Create a DataFrame and clean the data
reviews_df = pd.DataFrame(all_reviews)

# Clean and organize the DataFrame
if not reviews_df.empty:
    # First, let's see what columns we actually have
    print(f"Available columns: {reviews_df.columns.tolist()}")
    print(f"Sample data structure:")
    print(reviews_df.head(1).to_dict('records'))
    
    # Define the columns we want to keep (only if they exist)
    desired_columns = {
        'app_name': 'app_name',
        'review_date': 'review_date', 
        'score': 'rating',
        'content': 'review_text',
        'scrape_date': 'scrape_date'
    }
    
    # Optional columns (may or may not exist)
    optional_columns = {
        'thumbsUpCount': 'likes',
        'thumbsUp': 'likes',
        'repliedAt': 'replied_at',
        'reviewId': 'review_id',
        'userName': 'user_name'
    }
    
    # Select columns that exist
    columns_to_keep = []
    new_column_names = []
    
    # Add essential columns
    for orig_col, new_name in desired_columns.items():
        if orig_col in reviews_df.columns:
            columns_to_keep.append(orig_col)
            new_column_names.append(new_name)
    
    # Add optional columns (prefer thumbsUpCount over thumbsUp)
    for orig_col, new_name in optional_columns.items():
        if orig_col in reviews_df.columns and orig_col not in columns_to_keep:
            # Skip thumbsUp if we already have thumbsUpCount
            if orig_col == 'thumbsUp' and 'thumbsUpCount' in columns_to_keep:
                continue
            columns_to_keep.append(orig_col)
            new_column_names.append(new_name)
    
    # Select and rename columns
    reviews_df = reviews_df[columns_to_keep]
    reviews_df.columns = new_column_names
    
    # Convert review_date to datetime for proper sorting
    reviews_df['review_date'] = pd.to_datetime(reviews_df['review_date'])
    
    # Sort by date (newest first)
    reviews_df = reviews_df.sort_values('review_date', ascending=False)

# Save to CSV with timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'kenya_telco_app_reviews_{timestamp}.csv'
reviews_df.to_csv(filename, index=False, encoding='utf-8')

# Print summary statistics
print("\nCollection Summary:")
print("-" * 50)
print(f"Safaricom App reviews: {total_safaricom}")
print(f"Airtel Money App reviews: {total_airtel}")
print(f"Total reviews collected: {len(reviews_df)}")
if not reviews_df.empty:
    print(f"\nDate range: {reviews_df['review_date'].min().strftime('%Y-%m-%d')} to {reviews_df['review_date'].max().strftime('%Y-%m-%d')}")
    print(f"\nReviews by app:")
    print(reviews_df['app_name'].value_counts())
    print(f"\nAverage ratings:")
    print(reviews_df.groupby('app_name')['rating'].mean())
    
    # Print reviews by year
    print("\nReviews by year:")
    print(reviews_df.groupby([reviews_df['review_date'].dt.year, 'app_name']).size().unstack(fill_value=0))
print(f"\nData saved to: {filename}")